In [9]:
import polars as pl
import pandas as pd

data_dir = '../data/jp_morgan'
data_ = 'fraud'

sampling = False
if sampling:
    data_name = f'sample_{data_}'
else:
    data_name = f'final_{data_}'

In [10]:
df = pd.read_parquet(f'{data_dir}/sorted/transaction_metrics_{data_name}.parquet')

In [11]:
df

,Time_step,Label,Transaction_Id,Transaction_Type,std_txn_type,std_txn_method,is_credit,USD_amount,party_Id,party_Account,...,volume_30d_count,volume_avg_7d_to_14d_ratio,volume_count_7d_to_14d_ratio,volume_avg_7d_to_30d_ratio,volume_count_7d_to_30d_ratio,volume_avg_14d_to_30d_ratio,volume_count_14d_to_30d_ratio,stat_7d_velocity,stat_14d_velocity,stat_30d_velocity
0,2022-03-15 10:24:00,0,EXCHANGE-10115,WITHDRAWAL-EXCHANGE,WITHDRAWAL,CHECK,False,558.43,JPMC-CLIENT-10098,ACCOUNT-10108,...,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2022-03-15 10:24:00,0,QUICK-PAYMENT-10116,QUICK-PAYMENT-QUICK-PAYMENT,PAYMENT,ELECTRONIC,False,622.78,JPMC-CLIENT-10098,ACCOUNT-10109,...,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2022-03-15 10:24:00,0,QUICK-PAYMENT-1110,QUICK-PAYMENT-QUICK-PAYMENT,PAYMENT,ELECTRONIC,False,788.61,JPMC-CLIENT-1093,ACCOUNT-1104,...,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2022-03-15 10:24:00,0,MAKE-PAYMENT-1113,MAKE-PAYMENT-MAKE-PAYMENT,PAYMENT,ELECTRONIC,True,100.47,JPMC-CLIENT-1093,ACCOUNT-1103,...,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2022-03-15 10:24:00,0,CASH-CHECK-1399,EXCHANGE-CASH-CHECK,WITHDRAWAL,CHECK,False,663.68,JPMC-CLIENT-1379,ACCOUNT-1390,...,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1498172,2054-09-14 13:17:24,0,PAY-BILL-2180018,MAKE-PAYMENT-PAY-BILL,PAYMENT,ELECTRONIC,False,159.64,JPMC-CLIENT-2179746,ACCOUNT-2179757,...,98,1.030021,1.238095,0.954463,1.137026,0.926643,0.918367,1701.731429,1334.414286,1568.056000
1498173,2054-09-15 00:57:24,0,QUICK-PAYMENT-2181421,QUICK-PAYMENT-QUICK-PAYMENT,PAYMENT,ELECTRONIC,False,120.07,JPMC-CLIENT-2181147,ACCOUNT-2181158,...,96,1.068075,1.086957,1.094857,1.116071,1.025075,1.026786,1791.974286,1543.539286,1466.501000
1498174,2054-09-15 01:17:24,0,PAYMENT-2180021,MAKE-PAYMENT-PAYMENT,PAYMENT,ELECTRONIC,False,25.32,JPMC-CLIENT-2179746,ACCOUNT-2179757,...,96,1.004562,1.209302,0.970625,1.160714,0.966217,0.959821,1728.008571,1422.441429,1533.801333
1498175,2054-09-15 12:57:24,0,QUICK-PAYMENT-2181422,QUICK-PAYMENT-QUICK-PAYMENT,PAYMENT,ELECTRONIC,False,610.91,JPMC-CLIENT-2181147,ACCOUNT-2181158,...,96,1.068075,1.086957,1.094857,1.116071,1.025075,1.026786,1791.974286,1543.539286,1466.501000


In [12]:
def extract_entities(df):
    df1 = df.groupby('party_Id').agg({'party_Country': lambda x: pd.Series.mode(x)[0] if not pd.Series.mode(x).empty else None})
    party_df = df[['party_Id', 'party_Account']].join(df1, on='party_Id')
    party_df.columns = ['Id', 'Account', 'Country']

    df2 = df.groupby('cparty_Id').agg({'cparty_Country': lambda x: pd.Series.mode(x)[0] if not pd.Series.mode(x).empty else None})
    cparty_df = df[['cparty_Id', 'cparty_Account']].join(df2, on='cparty_Id')
    cparty_df.columns = ['Id', 'Account', 'Country']

    return pd.concat([party_df, cparty_df]).drop_duplicates()

entities = extract_entities(df)
entities.to_parquet(f'{data_dir}/sorted/entities_{data_name}.parquet')